# 1 Introduction and REST API

Objectives
 * Verify HDF Lab is working correctly
 * Learn how to make REST requests to the HSDS HDF Server

In [1]:
# this is the server endpoint 
hs_endpoint = "http://hsdshdflab.hdfgroup.org"   

In [2]:
# requests is a popular python library for making http requests
import requests
import json
# numpy is a popular array library
import numpy as np

In [3]:
# request server status info
rsp = requests.get(hs_endpoint + "/about")

In [4]:
rsp.status_code  # should be 200

200

In [5]:
# The response include the server state (hopefully 'READY')
# and other information about the service
rsp.json()

{'start_time': 1642258766,
 'state': 'READY',
 'hsds_version': '0.7.0beta',
 'name': 'Highly Scalable Data Service (HSDS) for HDFLab',
 'greeting': 'Welcome to HSDS!',
 'about': 'HSDS is a webservice for HDF data',
 'node_count': 4,
 'dn_urls': ['http://192.168.10.148:6101',
  'http://192.168.11.113:6101',
  'http://192.168.13.76:6101',
  'http://192.168.7.82:6101'],
 'dn_ids': ['dn-b1315', 'dn-d943f', 'dn-535d1', 'dn-9c81a'],
 'username': 'anonymous',
 'isadmin': False}

Problem: "/info" is another request supported by the server.  What does it return?

In [6]:
# get information about a HDF5 "file" on the server
params={"domain":"/shared/tall.h5"}
rsp = requests.get(hs_endpoint + "/", params=params)
rsp.status_code

200

In [7]:
# show the response contents
rsp_json = rsp.json()
rsp_json

{'root': 'g-d38053ea-3418fe27-5b08-db62bc-9076af',
 'class': 'domain',
 'owner': 'admin',
 'created': 1622930252.3698952,
 'limits': {'min_chunk_size': 1048576,
  'max_chunk_size': 4194304,
  'max_request_size': 104857600,
  'max_chunks_per_request': 1000},
 'compressors': ['blosclz', 'lz4', 'lz4hc', 'gzip', 'zstd'],
 'version': '0.7.0beta',
 'lastModified': 1623085764.3507726,
 'hrefs': [{'rel': 'self',
   'href': 'http://hsdshdflab.hdfgroup.org/?domain=/shared/tall.h5'},
  {'rel': 'database',
   'href': 'http://hsdshdflab.hdfgroup.org/datasets?domain=/shared/tall.h5'},
  {'rel': 'groupbase',
   'href': 'http://hsdshdflab.hdfgroup.org/groups?domain=/shared/tall.h5'},
  {'rel': 'typebase',
   'href': 'http://hsdshdflab.hdfgroup.org/datatypes?domain=/shared/tall.h5'},
  {'rel': 'root',
   'href': 'http://hsdshdflab.hdfgroup.org/groups/g-d38053ea-3418fe27-5b08-db62bc-9076af?domain=/shared/tall.h5'},
  {'rel': 'acls',
   'href': 'http://hsdshdflab.hdfgroup.org/acls?domain=/shared/tall.h5'

In [8]:
# The domain response includes a key identifying the root group
root_id = rsp_json["root"]
root_id

'g-d38053ea-3418fe27-5b08-db62bc-9076af'

In [9]:
# getinfo about the root group
rsp = requests.get(hs_endpoint + "/groups/" + root_id, params=params)
rsp.status_code

200

In [10]:
root_json = rsp.json()
root_json

{'id': 'g-d38053ea-3418fe27-5b08-db62bc-9076af',
 'root': 'g-d38053ea-3418fe27-5b08-db62bc-9076af',
 'created': 1622930252.2514794,
 'lastModified': 1622930252.985488,
 'linkCount': 2,
 'attributeCount': 2,
 'domain': '/shared/tall.h5',
 'bucket': 'hdflab2',
 'hrefs': [{'rel': 'self',
   'href': 'http://hsdshdflab.hdfgroup.org/groups/g-d38053ea-3418fe27-5b08-db62bc-9076af?domain=/shared/tall.h5'},
  {'rel': 'links',
   'href': 'http://hsdshdflab.hdfgroup.org/groups/g-d38053ea-3418fe27-5b08-db62bc-9076af/links?domain=/shared/tall.h5'},
  {'rel': 'root',
   'href': 'http://hsdshdflab.hdfgroup.org/groups/g-d38053ea-3418fe27-5b08-db62bc-9076af?domain=/shared/tall.h5'},
  {'rel': 'home',
   'href': 'http://hsdshdflab.hdfgroup.org/?domain=/shared/tall.h5'},
  {'rel': 'attributes',
   'href': 'http://hsdshdflab.hdfgroup.org/groups/g-d38053ea-3418fe27-5b08-db62bc-9076af/attributes?domain=/shared/tall.h5'}]}

Problem: replace the root_id in the last request with "g-1234".  What status code do you get?

In [11]:
# get the dataset at /g1/g1.1/dset1.1.1
# instead of using the dataset id, we'll provide the 
# path to the object within the HDF file as query parameter
params["h5path"] = "/g1/g1.1/dset1.1.1"
rsp = requests.get(hs_endpoint + "/datasets/", params=params)
rsp.status_code

200

In [12]:
dset_json = rsp.json()
dset_json

{'id': 'd-d38053ea-3418fe27-22d9-478e7b-913279',
 'root': 'g-d38053ea-3418fe27-5b08-db62bc-9076af',
 'shape': {'class': 'H5S_SIMPLE', 'dims': [10, 10], 'maxdims': [10, 10]},
 'type': {'class': 'H5T_INTEGER', 'base': 'H5T_STD_I32BE'},
 'creationProperties': {'layout': {'class': 'H5D_CHUNKED', 'dims': [10, 10]},
  'fillTime': 'H5D_FILL_TIME_ALLOC'},
 'layout': {'class': 'H5D_CHUNKED', 'dims': [10, 10]},
 'attributeCount': 2,
 'created': 1622930253,
 'lastModified': 1622930253,
 'domain': '/shared/tall.h5',
 'hrefs': [{'rel': 'self',
   'href': 'http://hsdshdflab.hdfgroup.org/datasets/d-d38053ea-3418fe27-22d9-478e7b-913279?domain=/shared/tall.h5'},
  {'rel': 'root',
   'href': 'http://hsdshdflab.hdfgroup.org/groups/g-d38053ea-3418fe27-5b08-db62bc-9076af?domain=/shared/tall.h5'},
  {'rel': 'home',
   'href': 'http://hsdshdflab.hdfgroup.org/?domain=/shared/tall.h5'},
  {'rel': 'attributes',
   'href': 'http://hsdshdflab.hdfgroup.org/datasets/d-d38053ea-3418fe27-22d9-478e7b-913279/attributes

In [13]:
# read all the values from the dataset
dset_id = dset_json["id"]
del params["h5path"]  # don't need this now
rsp = requests.get(hs_endpoint + "/datasets/" + dset_id + "/value", params=params)
rsp.status_code

200

In [14]:
value_json = rsp.json()
value_json["value"]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 2, 4, 6, 8, 10, 12, 14, 16, 18],
 [0, 3, 6, 9, 12, 15, 18, 21, 24, 27],
 [0, 4, 8, 12, 16, 20, 24, 28, 32, 36],
 [0, 5, 10, 15, 20, 25, 30, 35, 40, 45],
 [0, 6, 12, 18, 24, 30, 36, 42, 48, 54],
 [0, 7, 14, 21, 28, 35, 42, 49, 56, 63],
 [0, 8, 16, 24, 32, 40, 48, 56, 64, 72],
 [0, 9, 18, 27, 36, 45, 54, 63, 72, 81]]

In [15]:
# get just the upper 3x4 elements
# We use the select query param with a numpy like metod to 
# specify the sub-region we want
params["select"] = "[0:3, 0:4]"
rsp = requests.get(hs_endpoint + "/datasets/" + dset_id + "/value", params=params)
rsp.status_code

200

In [16]:
subarray_json = rsp.json()
subarray_json["value"]

[[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 4, 6]]

In [17]:
# Read elements (i,i) for i in range 0-9.
# This is a point selection where we are explictly
# giving the indices of the elements we want.
# Since the number of points could be quite large
# we use POST with the indices provided in the body of 
# the request, rather than GET
points = []
for i in range(10):
    points.append([i,i])
body = {"points": points}
del params["select"] # remove select so we will read across entire dataset
rsp = requests.post(hs_endpoint + "/datasets/" + dset_id + "/value", params=params, data=json.dumps(body))
rsp.status_code

200

In [18]:
# The returned array is always one-dimensional
points_json = rsp.json()
points_json["value"]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [19]:
# we can also get data values as a binary stream
headers = {"accept": "application/octet-stream"}
rsp = requests.post(hs_endpoint + "/datasets/" + dset_id + "/value", params=params, data=json.dumps(body), headers=headers)
rsp.status_code


200

In [20]:
rsp.content

b'\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x04\x00\x00\x00\t\x00\x00\x00\x10\x00\x00\x00\x19\x00\x00\x00$\x00\x00\x001\x00\x00\x00@\x00\x00\x00Q'

In [21]:
# we can create a numpy array object using this data
np.frombuffer(rsp.content, dtype='>i4')

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)

Problem: what would you guess the '>i4' means?  What happens if you use '<i4' instead?